In [12]:
from lightning import LightningModule, seed_everything, LightningDataModule, Callback, Trainer
from hydra_zen import builds, make_config, instantiate
from omegaconf import MISSING, DictConfig, OmegaConf

In [13]:
lit_module_config = builds(LightningModule, populate_full_signature=True)

In [16]:
x = lit_module_config()

In [19]:
print(x)

Builds_LightningModule(_target_='lightning.pytorch.core.module.LightningModule')


In [6]:
c = make_config(lit_module=builds(LightningModule, populate_full_signature=True))

In [11]:
c()

Config(lit_module=<class 'types.Builds_LightningModule'>)

In [ ]:
def pre_seed(seed:int):
    seed_everything(seed)

In [ ]:
def start_run(config):
    run = instantiate(config.run)
    with run(config=OmegaConf.to_container(OmegaConf.structured(config))) as run:
        return train(
            model=config.model, 
            datamodule=config.datamodule, 
            trainer=config.trainer,
            optim=config.optim,
            callbacks=config.callbacks,
            seed=config.seed,
            run=run,
            )

In [ ]:
def train(
    model, 
    datamodule, 
    trainer,
    optim, 
    run,
    callbacks,
    seed: int = 1234,
    
    ):
    
        
    datamodule: LightningDataModule = instantiate(datamodule)

    datamodule.prepare_data()
    datamodule.setup()
    
    dl = iter(datamodule.train_dataloader())
    batch = next(dl)

    model: LightningModule = instantiate(model)

    model.forward(batch)

    model.init_weights()


    trainer: Trainer = instantiate(trainer, callbacks=callbacks, _convert_="partial")
    
    trainer.fit(model=model, datamodule=datamodule)